# HW5 MPNN

## Overview

In this question, we will try the Graph Neural Network (GNN).

---

In [1]:
import os
import pickle
import json
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
DATA_PATH = "../HW5_GNN-lib/data/"

---

We will use [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/index.html), which is a geometric deep learning extension library for PyTorch.

## 1 Graph [10 points]

First, let us learn the fundamental concepts of PyTorch Geometric.

A graph is used to model pairwise relations (edges) between objects (nodes).
A single graph in PyTorch Geometric is described by an instance of `torch_geometric.data.Data`, which holds the following attributes by default:

- `data.x`: Node feature matrix with shape `[num_nodes, num_node_features]`
- `data.edge_index`: Graph connectivity in COO format with shape `[2, num_edges]` and type `torch.long`
- `data.edge_attr`: Edge feature matrix with shape `[num_edges, num_edge_features]`
- `data.y`: Target to train against (may have arbitrary shape), *e.g.*, node-level targets of shape `[num_nodes, *]` or graph-level targets of shape `[1, *]`
- `data.pos`: Node position matrix with shape `[num_nodes, num_dimensions]`

Note that none of these attributes is required.

We show a simple example of an unweighted and undirected graph with three nodes and four edges. Each node contains exactly one feature:

<img width="500" src="img/graph-1.png">

In [4]:
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
print(data)
print("num_nodes:", data.num_nodes)
print("num_edges:", data.num_edges)
print("num_node_features:", data.num_node_features)

Data(x=[3, 1], edge_index=[2, 4])
num_nodes: 3
num_edges: 4
num_node_features: 1


Note that although the graph has only two edges, we need to define four index tuples to account for both directions of a edge.

Now, create a `torch_geometric.data.Data` instance for the following graph.

<img width="250" src="img/graph-2.png">

Assign the graph-level target $1$ to the graph.

In [5]:
"""
TODO: create a `torch_geometric.data.Data` instance for the graph above. Set the graph-level target to 1.
"""
import torch
from torch_geometric.data import Data


# Define the edge index for the directed graph (without doubling the edges)
edge_index = torch.tensor([
    [0, 1],
    [1, 2],
    [1, 3],
    [2, 0]
], dtype=torch.long)

# Define the target for the graph
y = torch.tensor([1], dtype=torch.float)

# Create the Data instance
data = Data(edge_index=edge_index.t().contiguous(), y=y)

print(data)


Data(edge_index=[2, 4], y=[1])


In [6]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

assert data.num_nodes == 4
assert data.num_edges == 4
assert data.y == 1
assert data.num_node_features == 0
assert data.num_edge_features == 0



In [7]:
data.num_nodes, data.num_edges,data.y, data.num_node_features, data.num_edge_features

(4, 4, tensor([1.]), 0, 0)

## 2 Dataset [20 points]

For this question, we will use the [MUTAG dataset](http://networkrepository.com/Mutag.php). Each graph in the dataset represents a chemical compound and graph labels represent their mutagenic effect on a specific gram negative bacterium. The dataset includes 188 graphs. Graph nodes have 7 labels and each graph is labelled as belonging to 1 of 2 classes.

In [8]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root=DATA_PATH, name='MUTAG')
print("len:", len(dataset))
print("num_classes:", dataset.num_classes)
print("num_node_features:", dataset.num_node_features)

len: 188
num_classes: 2
num_node_features: 7


Let us take one graph as an example.

In [9]:
data = dataset[0]
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Number of features: {data.num_node_features}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of nodes: 17
Number of edges: 38
Number of features: 7
Average node degree: 2.24
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


We can see that the first graph in the dataset contains 17 nodes, each one having 7 features. There are 38/2 = 19 undirected edges and the graph is assigned to exactly one class. In addition, the data object is holding exactly one graph-level target.

In [10]:
def graph_stat(dataset):
    """
    TODO: calculate the statistics of the ENZYMES dataset.
    
    Outputs:
        min_num_nodes: min number of nodes
        max_num_nodes: max number of nodes
        mean_num_nodes: average number of nodes
        min_num_edges: min number of edges
        max_num_edges: max number of edges
        mean_num_edges: average number of edges
    """
    
    num_nodes = [data.num_nodes for data in dataset]
    num_edges = [data.num_edges for data in dataset]
    
    min_num_nodes = min(num_nodes)
    max_num_nodes = max(num_nodes)
    mean_num_nodes = sum(num_nodes) / len(num_nodes)
    
    min_num_edges = min(num_edges)
    max_num_edges = max(num_edges)
    mean_num_edges = sum(num_edges) / len(num_edges)
    
    return min_num_nodes, max_num_nodes, mean_num_nodes, min_num_edges, max_num_edges, mean_num_edges

In [11]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

assert np.allclose(graph_stat(dataset), (10, 28, 17.93, 20, 66, 39.58), atol=1e-2)



Neural networks are usually trained in a batch-wise fashion. PyTorch Geometric achieves parallelization over a mini-batch by creating sparse block diagonal adjacency matrices (defined by `edge_index`) and concatenating feature and target matrices in the node dimension. This composition allows differing number of nodes and edges over examples in one batch:

$\begin{split}\mathbf{A} = \begin{bmatrix} \mathbf{A}_1 & & \\ & \ddots & \\ & & \mathbf{A}_n \end{bmatrix}, \qquad \mathbf{X} = \begin{bmatrix} \mathbf{X}_1 \\ \vdots \\ \mathbf{X}_n \end{bmatrix}, \qquad \mathbf{Y} = \begin{bmatrix} \mathbf{Y}_1 \\ \vdots \\ \mathbf{Y}_n \end{bmatrix}\end{split}$

Luckily, PyTorch Geometric contains its own `torch_geometric.data.DataLoader`, which already takes care of this concatenation process. Let us learn about it in an example:

In [12]:
from torch_geometric.data import DataLoader

loader = DataLoader(dataset, batch_size=32, shuffle=True)

loader_iter = iter(loader)
batch = next(loader_iter)
print(batch)
print(batch.num_graphs)

DataBatch(edge_index=[2, 1260], x=[571, 7], edge_attr=[1260, 4], y=[32], batch=[571], ptr=[33])
32


That is, each batch contains $32$ graphs whose nodes and edges are stacked into one matrix.

Now, let us create a 80/20 train/test split, and load them into the dataloaders.

In [13]:
# shuffle
dataset = dataset.shuffle()
# split
split_idx = int(len(dataset) * 0.8)
train_dataset = dataset[:split_idx]
test_dataset = dataset[split_idx:]

print("len train:", len(train_dataset))
print("len test:", len(test_dataset))

len train: 150
len test: 38


In [14]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

## 3 Graph Neural Network [50 points]

After learning about the fundamental concepts in PyTorch Geometric, let us try implement our first graph neural network.

We will use a simple Graph Convolution Network (GCN) to assign each enzyme to one of the 6 EC top-level classes.

For a high-level explanation on GCN, have a look at its [blog](http://tkipf.github.io/graph-convolutional-networks/) post.

We will first implement a GCN layer. A GCN layer is given an adjacency matrix $A\in\mathbb{R}^{N\times N}$ and a node feature matrix $X\in\mathbb{R}^{N\times D_{in}}$, where $N$ is the number of nodes and $D_{in}$ is the input dimension. The graph convolution network will calculate its output by:

$$
X' = \hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}X\Theta
$$

where $\hat{A}=A+I$ denotes the adjacency matrix with self-loop, $\hat{D}_{ii}=\sum_{j=0}^{N-1}A_{ij}$ is its diagonal degree matrix and $\Theta\in\mathbb{R}^{D_{in}\times D_{out}}$ is the model parameter.

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import add_self_loops, degree
import math


class GCNConv(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__()
        self.theta = nn.Parameter(torch.FloatTensor(in_channels, out_channels))
        # Initialize the parameters.
        stdv = 1. / math.sqrt(out_channels)
        self.theta.data.uniform_(-stdv, stdv)
    
    def forward(self, x, edge_index):
        """
        TODO:
            1. Generate the adjacency matrix with self-loop \hat{A} using edge_index.
            2. Calculate the diagonal degree matrix \hat{D}.
            3. Calculate the output X' with torch.mm using the equation above.
        """
        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Compute the degree matrix.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        D_inv_sqrt = torch.diag(deg_inv_sqrt)

        # Step 3: Compute the normalized adjacency matrix.
        A_hat = torch.sparse_coo_tensor(edge_index, torch.ones(edge_index.size(1)), (x.size(0), x.size(0)))
        A_hat = A_hat.to_dense()
        A_hat = torch.mm(torch.mm(D_inv_sqrt, A_hat), D_inv_sqrt)

        # Step 4: Compute the output features.
        X_prime = torch.mm(A_hat, x)
        X_prime = torch.mm(X_prime, self.theta)

        return X_prime

In [16]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''



'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

<img width="500" src="img/graph-3.png">

The GCN will have the following steps:

- Embed each node by performing multiple rounds of message passing
- Aggregate node embeddings into a unified graph embedding (readout layer)
- Train a final classifier on the graph embedding


There exists multiple readout layers in literature, but the most common one is to simply take the average of node embeddings:

$$
\mathbf{x}_{\mathcal{G}} = \frac{1}{|\mathcal{V}|} \sum_{v \in \mathcal{V}} \mathcal{x}^{(L)}_v
$$

PyTorch Geometric provides this functionality via `torch_geometric.nn.global_mean_pool`.

In [17]:
# from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        """
        TODO:
            1. Define the first convolution layer using `GCNConv()`. Set `out_channels` to 64;
            2. Define the first activation layer using `nn.ReLU()`;
            3. Define the second convolution layer using `GCNConv()`. Set `out_channels` to 64;
            4. Define the second activation layer using `nn.ReLU()`;
            5. Define the third convolution layer using `GCNConv()`. Set `out_channels` to 64;
            6. Define the dropout layer using `nn.Dropout()`;
            7. Define the linear layer using `nn.Linear()`. Set `output_size` to 2.

        Note that for MUTAG dataset, the number of node features is 7, and the number of classes is 2.

        """
        
        self.conv1 = GCNConv(7, 64)
        self.relu1 = nn.ReLU()
        self.conv2 = GCNConv(64, 64)
        self.relu2 = nn.ReLU()
        self.conv3 = GCNConv(64, 64)
        self.dropout = nn.Dropout(p=0.5)
        self.linear = nn.Linear(64, 2)

    def forward(self, x, edge_index, batch):
        """
        TODO:
            1. Pass the data through the frst convolution layer;
            2. Pass the data through the activation layer;
            3. Pass the data through the second convolution layer;
            4. Obtain the graph embeddings using the readout layer with `global_mean_pool()`;
            5. Pass the graph embeddgins through the dropout layer;
            6. Pass the graph embeddings through the linear layer.
            
        Arguments:
            x: [num_nodes, 7], node features
            edge_index: [2, num_edges], edges
            batch: [num_nodes], batch assignment vector which maps each node to its 
                   respective graph in the batch

        Outputs:
            probs: probabilities of shape (batch_size, 2)
        """
        
        x = self.conv1(x, edge_index)
        x = self.relu1(x)
        x = self.conv2(x, edge_index)
        x = self.relu2(x)
        x = self.conv3(x, edge_index)

        # Obtain the graph embeddings using the readout layer
        x = global_mean_pool(x, batch)

        # Pass the graph embeddings through the dropout layer
        x = self.dropout(x)

        # Pass the graph embeddings through the linear layer
        x = self.linear(x)
        
        return x
        
GCN()

GCN(
  (conv1): GCNConv()
  (relu1): ReLU()
  (conv2): GCNConv()
  (relu2): ReLU()
  (conv3): GCNConv()
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=64, out_features=2, bias=True)
)

In [18]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''



'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

In [19]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''



'\nAUTOGRADER CELL. DO NOT MODIFY THIS.\n'

## 4 Training and Inferencing [20 points]

Let us train our network for a few epochs to see how well it performs on the training as well as test set.

In [ ]:
gcn = GCN()

# optimizer
optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)
# loss
criterion = torch.nn.CrossEntropyLoss()

def train(train_loader):
    gcn.train()
    for data in train_loader:  # Iterate in batches over the training dataset.
        """
        TODO: train the model for one epoch.
        
        Note that you can acess the batch data using `data.x`, `data.edge_index`, `data.batch`, `data,y`.
        """
        
        optimizer.zero_grad()  # Clear gradients.
        out = gcn(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.

def test(loader):
    gcn.eval()
    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = gcn(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(200):
    train(train_loader)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch + 1:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [ ]:
'''
AUTOGRADER CELL. DO NOT MODIFY THIS.
'''

test_acc = test(test_loader)

